In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import np_utils

import _pickle as pickle

## Some useful functions to ease the processings
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertForQuestionAnswering, TFBertModel, TFBertForNextSentencePrediction

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

import tensorflow as tf

from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, TFGPT2Model
# from tensorflow.keras.backend.tensorflow_backend import set_session
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
# set_session(sess)


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


In [ ]:
df = pd.read_csv('train.article.txt', sep = '\n', header = None)
df1 = pd.read_csv('train.title.txt', sep = '\n', header = None)
df.columns = ['article']
df['title'] = df1[0]

In [ ]:
save(df, 'train_gigaword')

In [ ]:
df = load('train_gigaword')

In [ ]:
df = df.iloc[:200000]

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

max_length = 64
max_length_out = 16
X = list(np.zeros(df.shape[0]))
X_type = list(np.zeros(df.shape[0]))
X_masks = list(np.zeros(df.shape[0]))
Y = list(np.zeros(df.shape[0]))

text_pairs = []
for index, line in tqdm(df.iterrows(), total = df.shape[0]):
    s1 = line['article']
    s2 = line['title']
    
    tokenized = tokenizer.encode_plus(str(s1), add_special_tokens = False, max_length = max_length, pad_to_max_length = False)
    answer = tokenizer.encode_plus(str(s2), add_special_tokens = False, max_length = max_length_out, pad_to_max_length = False)
    
    X[index] = tokenized['input_ids']
    Y[index] = answer['input_ids']

In [ ]:
df['inputs_ids'] = X
df['target'] = Y

In [ ]:
save(df, 'small_data_refined_gpt2')

In [ ]:
df = load('small_data_refined_gpt2')

In [ ]:
df.tail()

In [ ]:
X = list(df['inputs_ids'].values)
Y = list(df['target'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.1)

In [ ]:
X_train[0]

In [ ]:
def generator(X, y, batch_size = 64, max_len = 64):
    
    while True:
        
        batch_in = list(np.zeros(batch_size))
        batch_mask = list(np.zeros(batch_size))
        batch_types = list(np.zeros(batch_size))
        batch_y = list(np.zeros(batch_size))
        
        batch = np.random.randint(0, len(X), batch_size)
        
        for elt in range(len(batch)):
            i = batch[elt]
            contexte = X[i]
            summ = y[i]
            
            sentence = contexte
            sentence.extend(summ)
            types = list(np.zeros(len(sentence)))
            types.extend(list(np.ones(len(summ))))
            masks = list(np.ones(len(sentence) + len(summ)))
            
            if len(sentence) > max_len+1:
                sentence = sentence[-max_len-1:]
                types = types[-max_len-1:]
                masks = masks[-max_len-1:]
            else:
                while len(sentence) < max_len + 1:
                    sentence.append(0)
                    types.append(0)
                    masks.append(0)
            
            inputs_ids = sentence[:max_len]
            types = types[:max_len]
            masks = masks[:max_len]
            
            output_ids = sentence[1:]
            
            batch_in[elt] = inputs_ids
            batch_mask[elt] = masks
            batch_types[elt] = types
            
            output_ids = np_utils.to_categorical(output_ids, num_classes = 50257)
            
            batch_y[elt] = np.array(output_ids)
        
        batch_x = [np.array(batch_in), np.array(batch_mask), np.array(batch_types)]
        
        yield( batch_x, np.array(batch_y) )

In [ ]:
a = generator(X_train, y_train, batch_size = 2, max_len = 64)

In [ ]:
b = a.send(None)

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
tokenizer.decode(b[0][0][0])

In [ ]:
tokenizer.decode(np.argmax(b[1][0], axis = 1))

In [ ]:
b[1].shape

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, TimeDistributed, LSTM

max_length = 64
vocab_size = 50257

inputs_ids = Input(shape = (max_length,), dtype = 'int32')
inputs_mask = Input(shape = (max_length,), dtype = 'int32')
inputs_type = Input(shape = (max_length,), dtype = 'int32')

inputs = [inputs_ids, inputs_mask, inputs_type]

sentence_encoder = TFGPT2Model.from_pretrained('gpt2')

encoded = sentence_encoder(inputs_ids, attention_mask = inputs_mask, token_type_ids = inputs_type)

encoded = encoded[0]

decoded = Dense(vocab_size, activation='softmax')(encoded)

model = Model(inputs, decoded)

InternalError: Blas GEMM launch failed : a.shape=(15, 768), b.shape=(768, 2304), m=15, n=2304, k=768 [Op:MatMul]

In [ ]:
model.summary()

In [ ]:
encoded

In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
import tensorflow as tf

loss_classif     =  'categorical_crossentropy'# find the right loss for multi-class classification
optimizer        =  Adam(3e-5, 1e-8) # find the right optimizer
metrics_classif  =  ['accuracy']


model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)

In [ ]:
X_train[0]

In [ ]:
batch_size = 16
epochs = 4

with tf.device('/GPU:0'):
    history = model.fit(generator(X_train, y_train, batch_size = batch_size, max_len = max_length), steps_per_epoch=len(X_train) // batch_size,
        epochs=epochs)

In [ ]:
model.save_weights('text_summary_gtp')

In [ ]:
model.load_weights('text_summary_gtp')

In [ ]:
dir(model)

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

In [ ]:
import tensorflow as tf

In [ ]:
tf.device('XLA_GPU:0')

In [ ]:
get_available_gpus()

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
def compute_next(inputs,masks,types, repetition_penalty = 1):
    input_1 = deepcopy(inputs)
    mask_1 = deepcopy(masks)
    types_1 = deepcopy(types)
        
    while len(input_1) < 64:
        input_1.append(0)
        mask_1.append(0)
        types_1.append(0)
        
    inp = [tf.constant(input_1)[None, :] , tf.constant(mask_1)[None, :] , tf.constant(types_1)[None, :] ]
        
    out = model.predict(inp)
        
    out = model.predict(inp)[0][len(inputs)-1]
        
    if repetition_penalty != 1 and masks[len(inputs)-1] != 0:
        for elt in inputs:
            if out[elt] < 0:
                out[elt] *= repetition_penalty
            else:
                out[elt] /= repetition_penalty
    return out

In [ ]:
from copy import deepcopy
def predict(a, repetition_penalty = 1, beam_search = 1, beam_width = 5):
    
    inputs = deepcopy(a)
    masks = list(np.ones(len(inputs)))
    types = list(np.zeros(len(inputs)))
    
    inputs.append(inputs[0])
    masks.append(1)
    types.append(1)
    
    if len(inputs)>45:
        inputs = inputs[-45:]
        masks = masks[-45:]
        types = types[-45:]
        
    while len(inputs) < 64:
        print(len(inputs))
        
        out = compute_next(inputs,masks,types, repetition_penalty = repetition_penalty)
        
        decrypt = np.argmax(out)
        
        if beam_search != 1 and len(inputs) + beam_search < 64:
            current_depth = 1
            
            args = np.argsort(out)[-beam_width:]
            score = np.sort(out)[-beam_width:]
            
            for i, elt in enumerate(args):
                out1 = compute_next(inputs + [elt],masks + [1],types + [1], repetition_penalty = 1)
                score[i] *= np.max(out1)
            decrypt = args[np.argmax(score)]
            
            
        
        
        if decrypt == 0:
            inputs.append(decrypt)
            masks.append(0)
            types.append(0)
        else:
            inputs.append(decrypt)
            masks.append(1)
            types.append(1)
    return inputs, types, inputs

In [ ]:
a

In [ ]:
np.array(a)[np.argsort(a)[-5:]]

In [ ]:
ind = 72
a = X_test[ind]
b = y_test[ind]

In [ ]:
a= tokenizer.encode("Aries is the ninth studio album by Mexican recording artist Luis Miguel (pictured). Released by WEA Latina in 1993, it features pop ballads and dance numbers with R&B influences. 'Ayer' and 'Hasta Que Me Olvides', two of the album's three singles released for sale, topped the US Billboard Hot Latin Songs chart. 'Hasta el Fin' and 'Tú y Yo' were released as promotional singles")

In [ ]:
pred, types, inputs = predict(a, repetition_penalty = 2, beam_search = 1, beam_width = 5)

In [ ]:
print(tokenizer.decode(a))
print('\n')
print(tokenizer.decode(b))

In [ ]:
tokenizer.decode(np.array(pred)[np.array(types)==1])

In [ ]:
pred, types, inputs = predict(a, repetition_penalty = 2, beam_search = 2, beam_width = 5)

In [ ]:
tokenizer.decode(np.array(pred)[np.array(types)==1])

In [ ]:
tokenizer.decode(inputs)

In [ ]:
from copy import deepcopy
inputs = a
masks = list(np.ones(len(a)).astype(int))
types = list(np.zeros(len(a)).astype(int))
    
#     while len(inputs < 64):
        
input_1 = deepcopy(inputs)
mask_1 = deepcopy(masks)
types_1 = deepcopy(types)
        
while len(input_1) < 64:
    input_1.append(0)
    mask_1.append(0)
    types_1.append(0)
        
inp = [tf.constant(input_1)[None, :] , tf.constant(mask_1)[None, :] , tf.constant(types_1)[None, :] ]
        
out = model.predict(inp)[0]

In [ ]:
out.shape

In [ ]:
a

In [ ]:
len(a)

In [ ]:
np.argmax(out, axis = 1)[33]

In [ ]:
dir(model)

In [ ]:
inp

In [ ]:
np.argmax(a.send(None)[1], axis = 2)

In [ ]:
a = generator(X_test, y_test, batch_size = 1)
batch = a.send(None)
x = batch[0]
y = batch[1]
pred = model.predict(x)

In [ ]:
def encode(x):
    return np_utils.to_categorical(x, num_classes = 29611)

In [ ]:
x[3]

In [ ]:
np.array([encode([101,0,0,0,0,0,0,0,0,0,0])])

In [ ]:
x[3].shape

In [ ]:
np.argmax(x[3], axis = 2)[0]

In [ ]:
tok_pred = np.argmax(pred, axis = 2)[0]

In [ ]:
tok_true = np.argmax(y, axis = 2)[0]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
print(tokenizer.decode(x[0][0]))
print('\n')
print(tokenizer.decode(tok_true))
print('\n')
print(tokenizer.decode(tok_pred))
print('\n')